# Deep Double Descent
Autor: Piotr Branewski

## Założenia
1. Zaznajomić się z problemem deep double descent, opisanym na slajdach powyżej i cytowanym na slajdzie tytułowym artykule.
2. Wybrać dowolny zbiór danych o niewielkim wymiarze (M~10) i próbkach (wektorach) o współrzędnych (cechach) numerycznych. Liczność zbioru też bez przesady M~5000, .dane wieloklasowe. Zbiór testowy ok 20%. Proszę by zbiory się nie powtarzały by otrzymać lepszą statystykę.
3. Skupiamy się na architekturze sieci MLP (vanilla NN ze standardowymi parametrami) i zakładamy, że ilość parametrów sieci jest stała: (a) mniejsza od ilości danych; b) w przybliżeniu równa c) dużo większa.


### Przygotowanie datasetu

import libs

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

load dataset "Letter Recognition"

- Features: 16
- Samples: use a random subset of 5000 for manageable size
- Classes: 26 letter labels

In [2]:
letter = fetch_openml('letter', version=1, as_frame=False)
X_full, y_full = letter.data, letter.target
np.random.seed(42)
indices = np.random.choice(len(X_full), 5000, replace=False)
X, y = X_full[indices], y_full[indices]

#Function to get a fresh train/test split
def get_split(random_state):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=random_state
    )
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

### Zadanie 1
Dla KAŻDEJ z trzech sieci o różnej złożoności, zbadać jak zachowują się (ACC, F1) dla jej różnych realizacji czyli: a) sieć z jedną warstwą ukrytą, 2) dwoma 3) trzema itd.. Każdy wynik powtórzyć co najmniej 3 razy i wyliczyć średnią. Dla uproszczenia możemy przyjąć że ilość neuronów w warstwach ukrytych jest taka sama. Zakładamy, że CAŁKOWITA liczba neuronów jest taka sama (jak w założeniu 3)


In [3]:
# task 1
def build_mlp(n_layers, total_neurons, random_state=None):
    neurons = max(total_neurons // n_layers, 1)
    hidden_layer_sizes = tuple([neurons] * n_layers)
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation='relu',
        solver='adam',
        learning_rate='adaptive',
        learning_rate_init=1e-3,
        max_iter=10000,
        tol=1e-4,
        random_state=random_state,
        verbose=False
    )
    return model

results = []
total_neurons_list = [10, 20, 50]
for total_neurons in total_neurons_list:
    for n_layers in [1, 2, 3]:
        for repeat in range(3):
            X_train, X_test, y_train, y_test = get_split(random_state=repeat)
            model = build_mlp(n_layers, total_neurons, random_state=repeat)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='macro')
            results.append({
                'total_neurons': total_neurons,
                'n_layers': n_layers,
                'repeat': repeat,
                'accuracy': acc,
                'f1_macro': f1
            })

results

In [4]:
results_df = pd.DataFrame(results)
summary = results_df.groupby(['total_neurons', 'n_layers'])[['accuracy', 'f1_macro']].agg(['mean', 'std']).reset_index()
print(summary)

  total_neurons n_layers  accuracy            f1_macro          
                              mean       std      mean       std
0            10        1  0.784000  0.021000  0.783222  0.020384
1            10        2  0.631333  0.011590  0.627433  0.012280
2            10        3  0.340667  0.119182  0.305594  0.141511
3            20        1  0.840000  0.010392  0.838942  0.010538
4            20        2  0.777000  0.007211  0.776641  0.006894
5            20        3  0.686667  0.025541  0.688821  0.028471
6            50        1  0.888000  0.013892  0.887846  0.013465
7            50        2  0.867000  0.004583  0.866333  0.004117
8            50        3  0.840000  0.016643  0.839137  0.017208


### Wnioski z zadania 1
Zgodnie z przedstawionymi wynikami, dla ustalonej całkowitej liczby neuronów, zwiększanie liczby warstw (co prowadzi do zmniejszenia liczby neuronów w pojedynczej warstwie) generalnie pogarszało wydajność modelu (zarówno metrykę accuracy, jak i F1-macro). Efekt ten był szczególnie wyraźny przy mniejszej całkowitej liczbie neuronów (np. 10 neuronów). Przy większej liczbie neuronów (np. 50) spadek wydajności był mniej stromy, ale nadal zauważalny.

Kluczowe obserwacje:

- 10 neuronów:

    1 warstwa: Accuracy ~0.78
    
    3 warstwy: Accuracy ~0.34
- 50 neuronów:

    1 warstwa: Accuracy ~0.88
    
    3 warstwy: Accuracy ~0.84
Sugeruje to, że zbyt "cienkie" warstwy, nawet jeśli jest ich więcej, mogą nie być w stanie efektywnie przetwarzać informacji.



### Zadanie 2
Co zaobserwujemy zwiększając liczbę warstw przy ustalonej ilości parametrów sieci? Wysnuć hipotezę dlaczego tak się może dziać i czy istnieje jakiś element architektury którego zmiana zmieniłaby tą sytuację. Rekomenduję wykorzystanie obliczenia saturacji w poszczególnych warstwach.


In [5]:
# task 2
def get_hidden_activations(model, X):
    hidden_layer_sizes = model.hidden_layer_sizes
    if not hasattr(hidden_layer_sizes, '__iter__'):
        hidden_layer_sizes = [hidden_layer_sizes]
    
    activations_list = []
    current_input = X
    
    for i in range(model.n_layers_ - 2):
        raw_activation = current_input @ model.coefs_[i] + model.intercepts_[i]
        
        if model.activation == 'relu':
            activated_output = np.maximum(0, raw_activation)
        elif model.activation == 'tanh':
            activated_output = np.tanh(raw_activation)
        elif model.activation == 'logistic':
            activated_output = 1 / (1 + np.exp(-raw_activation))
        elif model.activation == 'identity':
            activated_output = raw_activation
        else:
            raise ValueError(f"Unsupported activation function: {model.activation}")
            
        activations_list.append(activated_output)
        current_input = activated_output
            
    return activations_list

def calculate_relu_saturation(model, X):
    if model.activation != 'relu':
        return [np.nan] * len(model.hidden_layer_sizes if hasattr(model.hidden_layer_sizes, '__iter__') else [model.hidden_layer_sizes])

    hidden_activations = get_hidden_activations(model, X)
    
    saturations_per_layer = []
    for layer_acts in hidden_activations:
        mean_layer_saturation = np.mean(layer_acts == 0)
        saturations_per_layer.append(mean_layer_saturation)
        
    return saturations_per_layer

saturation_analysis_results = []
total_neurons_configs_for_sat = [10, 50] 
n_layers_configs_for_sat = [1, 2, 3]

print("Calculating saturation for selected configurations...")
for total_neurons in total_neurons_configs_for_sat:
    for n_layers in n_layers_configs_for_sat:
        avg_sats_for_config_repeats = []
        detailed_sats_for_config_repeats = [] 
        
        for repeat in range(3):
            print(f"  Total neurons: {total_neurons}, Layers: {n_layers}, Repeat: {repeat}")
            X_train_sat, X_test_sat, y_train_sat, _ = get_split(random_state=repeat)
            
            if total_neurons // n_layers < 1 and n_layers > 0 :
                 print(f"    Skipping {total_neurons} total neurons / {n_layers} layers as neurons per layer < 1 before correction.")
                 continue

            model_for_saturation = build_mlp(n_layers, total_neurons, random_state=repeat)
            model_for_saturation.fit(X_train_sat, y_train_sat)
            
            layer_saturations = calculate_relu_saturation(model_for_saturation, X_test_sat)
            
            saturation_analysis_results.append({
                'total_neurons': total_neurons,
                'n_layers': n_layers,
                'repeat': repeat,
                'layer_saturations': layer_saturations,
                'mean_network_saturation': np.mean(layer_saturations) if layer_saturations else np.nan 
            })

saturation_df = pd.DataFrame(saturation_analysis_results)

print("\nSaturation Analysis Summary (Mean Network Saturation - averaged over layers and repeats):")
summary_saturation = saturation_df.groupby(['total_neurons', 'n_layers'])['mean_network_saturation'].agg(['mean', 'std']).reset_index()
print(summary_saturation)

print("\nDetailed Per-Layer Saturation (example for one repeat, Total Neurons=50):")
for n_layers_val in n_layers_configs_for_sat:
    detail = saturation_df[
        (saturation_df['total_neurons'] == 50) & 
        (saturation_df['n_layers'] == n_layers_val) &
        (saturation_df['repeat'] == 0)
    ]
    if not detail.empty:
        print(f"  Total Neurons: 50, Layers: {n_layers_val}, Repeat 0, Per-Layer Saturations: {detail['layer_saturations'].values[0]}")

print("\nDetailed Per-Layer Saturation (example for one repeat, Total Neurons=10):")
for n_layers_val in n_layers_configs_for_sat:
    detail = saturation_df[
        (saturation_df['total_neurons'] == 10) & 
        (saturation_df['n_layers'] == n_layers_val) &
        (saturation_df['repeat'] == 0)
    ]
    if not detail.empty:
        neurons_per_layer = max(10 // n_layers_val, 1)
        print(f"  Total Neurons: 10, Layers: {n_layers_val} ({neurons_per_layer} neurons/layer), Repeat 0, Per-Layer Saturations: {detail['layer_saturations'].values[0]}")

Calculating saturation for selected configurations...
  Total neurons: 10, Layers: 1, Repeat: 0
  Total neurons: 10, Layers: 1, Repeat: 1
  Total neurons: 10, Layers: 1, Repeat: 2
  Total neurons: 10, Layers: 2, Repeat: 0
  Total neurons: 10, Layers: 2, Repeat: 1
  Total neurons: 10, Layers: 2, Repeat: 2
  Total neurons: 10, Layers: 3, Repeat: 0
  Total neurons: 10, Layers: 3, Repeat: 1
  Total neurons: 10, Layers: 3, Repeat: 2
  Total neurons: 50, Layers: 1, Repeat: 0
  Total neurons: 50, Layers: 1, Repeat: 1
  Total neurons: 50, Layers: 1, Repeat: 2
  Total neurons: 50, Layers: 2, Repeat: 0
  Total neurons: 50, Layers: 2, Repeat: 1
  Total neurons: 50, Layers: 2, Repeat: 2
  Total neurons: 50, Layers: 3, Repeat: 0
  Total neurons: 50, Layers: 3, Repeat: 1
  Total neurons: 50, Layers: 3, Repeat: 2

Saturation Analysis Summary (Mean Network Saturation - averaged over layers and repeats):
   total_neurons  n_layers      mean       std
0             10         1  0.298700  0.036386
1    

### Wnioski z zadania 2
Obliczenia saturacji dla neuronów ReLU (mierzonej jako odsetek neuronów zwracających zero) dostarczyły dodatkowych informacji:

Dla 50 neuronów ogółem:

    1 warstwa: saturacja ~0.41
    
    2 warstwy: saturacje ~[0.42, 0.23]
    
    3 warstwy: saturacje ~[0.35, 0.23, 0.20]
    
Dla 10 neuronów ogółem:

    1 warstwa (10 neuronów/warstwę): saturacja ~0.27
    
    2 warstwy (5 neuronów/warstwę): saturacje ~[0.25, 0.07]
    
    3 warstwy (3 neurony/warstwę): saturacje ~[0.43, 0.33, 0.006]

***Interpretacja:***

W przypadku 50 neuronów, średnia saturacja w warstwach wydaje się maleć lub utrzymywać na podobnym poziomie w głębszych warstwach, co może oznaczać, że neurony pozostają aktywne. Jednak w konfiguracji z 10 neuronami i 3 warstwami, pierwsza warstwa wykazuje stosunkowo wysoką saturację (~0.43), co może wskazywać na "umieranie" neuronów i utratę informacji już na wczesnym etapie, co koreluje z drastycznym spadkiem wydajności obserwowanym w Zadaniu 1. Ostatnia warstwa (3 neurony) w tym przypadku wykazuje bardzo niską saturację (~0.006), co może oznaczać, że te nieliczne neurony są bardzo aktywne, ale jest ich zbyt mało, by efektywnie przetworzyć dane. Ogólnie, głębsze i cieńsze sieci mogą być bardziej podatne na problemy z przepływem informacji i aktywnością neuronów, co przekłada się na gorsze wyniki.

### Zadanie 3
W kontekście deep double descent proszę zobaczyć co dzieje się gdy analizujemy wyniki z punktu (2) dla 3 sieci o różnych złożonościach z zaszumionymi etykietami.

In [6]:
# task 3
def add_label_noise(y_train_orig, noise_fraction, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
        
    y_train_noisy = np.copy(y_train_orig)
    unique_labels = np.unique(y_train_orig)
    n_labels = len(y_train_noisy)
    n_to_flip = int(noise_fraction * n_labels)
    
    if n_to_flip == 0:
        return y_train_noisy

    flip_indices = np.random.choice(np.arange(n_labels), size=n_to_flip, replace=False)
    
    for idx in flip_indices:
        original_label = y_train_noisy[idx]
        possible_new_labels = [l for l in unique_labels if l != original_label]
        if not possible_new_labels:
            continue 
        new_label = np.random.choice(possible_new_labels)
        y_train_noisy[idx] = new_label
        
    return y_train_noisy

noise_level = 0.2
results_noisy = []
total_neurons_list_task3 = [10, 20, 50]
n_layers_list_task3 = [1, 2, 3]

print(f"\nRunning experiments with {noise_level*100}% label noise...")

for total_neurons in total_neurons_list_task3:
    for n_layers in n_layers_list_task3:
        acc_repeats = []
        f1_repeats = []
        for repeat in range(3): # 3 repeats
            print(f"  Total neurons: {total_neurons}, Layers: {n_layers}, Repeat: {repeat} (with noise)")

            X_train, X_test, y_train_clean, y_test_clean = get_split(random_state=repeat)
            
            y_train_noisy = add_label_noise(y_train_clean, noise_level, random_state=(42 + repeat))
            
            model_noisy = build_mlp(n_layers, total_neurons, random_state=repeat)
            model_noisy.fit(X_train, y_train_noisy)
            
            y_pred_noisy = model_noisy.predict(X_test)
            
            acc = accuracy_score(y_test_clean, y_pred_noisy)
            f1 = f1_score(y_test_clean, y_pred_noisy, average='macro', zero_division=0)
            
            results_noisy.append({
                'total_neurons': total_neurons,
                'n_layers': n_layers,
                'repeat': repeat,
                'noise_level': noise_level,
                'accuracy': acc,
                'f1_macro': f1
            })

results_noisy_df = pd.DataFrame(results_noisy)

print("\nSummary of Results with Noisy Labels:")
summary_noisy = results_noisy_df.groupby(['total_neurons', 'n_layers', 'noise_level'])[['accuracy', 'f1_macro']].agg(['mean', 'std']).reset_index()
print(summary_noisy)


Running experiments with 20.0% label noise...
  Total neurons: 10, Layers: 1, Repeat: 0 (with noise)
  Total neurons: 10, Layers: 1, Repeat: 1 (with noise)
  Total neurons: 10, Layers: 1, Repeat: 2 (with noise)
  Total neurons: 10, Layers: 2, Repeat: 0 (with noise)
  Total neurons: 10, Layers: 2, Repeat: 1 (with noise)
  Total neurons: 10, Layers: 2, Repeat: 2 (with noise)
  Total neurons: 10, Layers: 3, Repeat: 0 (with noise)
  Total neurons: 10, Layers: 3, Repeat: 1 (with noise)
  Total neurons: 10, Layers: 3, Repeat: 2 (with noise)
  Total neurons: 20, Layers: 1, Repeat: 0 (with noise)
  Total neurons: 20, Layers: 1, Repeat: 1 (with noise)
  Total neurons: 20, Layers: 1, Repeat: 2 (with noise)
  Total neurons: 20, Layers: 2, Repeat: 0 (with noise)
  Total neurons: 20, Layers: 2, Repeat: 1 (with noise)
  Total neurons: 20, Layers: 2, Repeat: 2 (with noise)
  Total neurons: 20, Layers: 3, Repeat: 0 (with noise)
  Total neurons: 20, Layers: 3, Repeat: 1 (with noise)
  Total neurons: 2

### Wnioski z zadania 3
Wprowadzenie 20% szumu do etykiet treningowych spowodowało oczekiwany spadek wydajności (accuracy i F1-macro) we wszystkich testowanych konfiguracjach w porównaniu do wyników z Zadania 1.

10 neuronów, 1 warstwa:

    Accuracy spadło z ~0.78 do ~0.73
10 neuronów, 3 warstwy:
    
    Accuracy spadło z ~0.34 do ~0.27
50 neuronów, 1 warstwa:
    
    Accuracy spadło z ~0.88 do ~0.82
50 neuronów, 3 warstwy:
    
    Accuracy spadło z ~0.84 do ~0.78
Tendencja obserwowana w Zadaniu 1 (pogarszanie się wyników wraz ze wzrostem liczby warstw przy stałej liczbie neuronów) została zachowana, a nawet w niektórych przypadkach procentowy spadek wydajności był bardziej dotkliwy. Modele trenowane na zaszumionych danych miały trudniejsze zadanie generalizacji, a ograniczenia związane ze zbyt cienkimi warstwami stały się jeszcze bardziej widoczne. Nie zaobserwowano wyraźnego zjawiska "deep double descent" w kontekście zmiany liczby warstw dla ustalonej liczby neuronów, raczej ogólne pogorszenie zdolności generalizacji przez modele.